# 웹 크롤링 및 텍스트 파싱을 이용한 성경 코퍼스 수집
파이썬을 이용하여 웹 크롤링(Crawling)을 구현한다. 크롤링으로 가져온 html데이터에서 텍스트 파싱하여 활용하고자하는 정보를 선별하여 추출한다.



성경(聖經)은 통상 기독교 또는 유대교 신앙의 최고 경전을 일컫는다. 성경은 구약성경과 신약성경으로 이루어져 있으며 각각 히브리어와 그리스어로 씌인 원본을 번역한 형태로 각 국가에서 사용하고 있다. 대한민국의 경우 2015년 현재 가장 많이 사용하는 역본은 ‘개역개정 4판(대한성서공회)’, ‘표준새번역(대한성서공회)’ 등이 있으며 그 밖에도 ‘현대인의 성경(생명의 말씀사)’, ‘우리말 성경(두란노)’, ‘메세지 성경(복있는 사람)’ 등의 다양한 역본이 존재한다. 이렇게 다양한 역본은 각각 출판사의 저작물로 등록되어 책으로 출판되고, 최근에는 스마트폰 어플리케이션 형태로 출시되고있다.

온라인에서도 각 성경을 역본별로 열람할 수 있다. 이러한 서비스를 제공하는 홈페이지는 대표적으로 Holy-Bible 다국어 성경( http://www.holybible.or.kr ), YouVersion( https://www.bible.com/ko ), 대한성서공회(http://www.bskorea.or.kr )이 있으며, (사)정보통신선교회 약정에 의해 컨텐츠에 대한 저작권을 명시한 채 공유되고 있다. 크롤링 사용은 저작권과 관련하여 민감한 소지가 있다. 이러한 방법을 유포하여 이득을 취하려는 행위는 저작권법에 의거 처벌될 수 있으므로, 단지 개인 연구목적용으로만 사용할 것이며, 이미 저작권 보호 기간이 만료된 ‘개역 한글판’에 대하여 진행한다.


In [1]:
class ReadingBible(object):
    '''
    classdocs
    '''
    bible_info = {
         "class" : "",                    # Old|New
         "bible_name_kr" : "",            # 창세기
         "bible_name_abbr_kr" : "",       # 창
         "bible_name_en" : "",            # Genesis
         "bible_name_abbr_en" : "",       # GEN
         "bskorea_name" : "",             # gen
         "chapter" : "",
         "from_section" : "",
         "to_section" : ""
        }
    bible_chapter_contents = {}

    def __init__(self, work_dir, bible_paragraph_info):
        '''
        성경  읽기를 위해 성경구절정보를 성경, 장, 절 정보로 속성 정의
        '''
        self.logger = logging.getLogger(__name__)

        # 사용할 클래스 내부 변수 선언
        self.bible_name_abbr_kr = ""
        self.chapter = ""
        self.from_section = ""
        self.to_section = ""
        
        # 셩경 약어로 성경정보 알아내는 성경정보 JSON파일 경로 설정
        # 성경 크롤링 결과 코퍼스 저장소 : 
        self.bible_abbr_file_name = os.path.join(work_dir, "../../data/bible/bible_abbr.json")

        if bible_paragraph_info:
            # '창 1:1~10'과 bible_paragraph_info에서  성경 약어 분리
            first_processing_list = bible_paragraph_info.split(' ')
            self.bible_name_abbr_kr = first_processing_list[0]
        
            # '1:1~10'과 bible_paragraph_info의 성경 장 절 정보에서 장 분리  
            second_processing_list = first_processing_list[1].split(':')
            self.chapter = second_processing_list[0]
        
        
            if '~' in second_processing_list[1]:
                third_processing_list = second_processing_list[1].split('~')
                self.from_section = third_processing_list[0]
                self.to_section = third_processing_list[1]
            elif '∼' in second_processing_list[1]:
                third_processing_list = second_processing_list[1].split('∼')
                self.from_section = third_processing_list[0]
                self.to_section = third_processing_list[1]
            else:
                self.from_section = second_processing_list[1]

    def get_bible_paragraph_info(self, bible_paragraph_info):
        '''
        성경구절을 분해하여 성경, 장, 절 정보로 분류 하여  Return
        '''
        paragraph_info = {
            "bible_name_abbr_kr" : "",  # 성경 한글 약어명
            "chapter" : "",             # 성경 장
            "from_section" : "",        # 시작구절
            "to_section" : ""           # 종료구절
        }
        # '창 1:1~10'과 bible_paragraph_info에서  성경 약어 분리
        first_processing_list = bible_paragraph_info.split(' ')
        paragraph_info['bible_name_abbr_kr'] = first_processing_list[0]
        
        # '1:1~10'과 bible_paragraph_info의 성경 장 절 정보에서 장 분리  
        second_processing_list = first_processing_list[1].split(':')
        paragraph_info['chapter'] = second_processing_list[0]
        
        
        if '~' in second_processing_list[1]:
            third_processing_list = second_processing_list[1].split('~')
            paragraph_info['from_section'] = third_processing_list[0]
            paragraph_info['to_section'] = third_processing_list[1]
        elif '∼' in second_processing_list[1]:
            third_processing_list = second_processing_list[1].split('∼')
            paragraph_info['from_section'] = third_processing_list[0]
            paragraph_info['to_section'] = third_processing_list[1]
        else:
            paragraph_info['from_section'] = second_processing_list[1]
            
        return paragraph_info
            
    def set_bible_paragraph_info(self, bible_paragraph_info):
        '''
        성경구절을 분해하여 성경, 장, 절 정보로 분류 
        '''
        # '창 1:1~10'과 bible_paragraph_info에서  성경 약어 분리
        first_processing_list = bible_paragraph_info.split(' ')
        self.bible_name_abbr_kr = first_processing_list[0]
        ReadingBible.bible_info['bible_name_abbr_kr'] = self.bible_name_abbr_kr
        
        # '1:1~10'과 bible_paragraph_info의 성경 장 절 정보에서 장 분리  
        second_processing_list = first_processing_list[1].split(':')
        self.chapter = second_processing_list[0]
        
        
        if '~' in second_processing_list[1]:
            third_processing_list = second_processing_list[1].split('~')
            self.from_section = third_processing_list[0]
            self.to_section = third_processing_list[1]
        elif '∼' in second_processing_list[1]:
            third_processing_list = second_processing_list[1].split('∼')
            self.from_section = third_processing_list[0]
            self.to_section = third_processing_list[1]
        else:
            self.from_section = second_processing_list[1]
            
    def get_query_info(self):
        '''
        대한 성서 공회 개역한글 성경을 크롤링하기위한 질의정보 생성
        '''
        query_info = {
            "bskorea_name" : "", # 대한성서공회에서 사용한 성경 영문약어명
            "chapter" : "",      # 성경 장
            "from_section" : "", # 시작구절
            "to_section" : ""    # 종료구절
        }
        
        # 요청한 성경 약어 획득
        
        # 성경영문약어를 Key로 성경이름정보를 정의한 JSON파일을 읽어들임
        with open(self.bible_abbr_file_name, 'rt', encoding='UTF8') as json_data:
            biblical_abbreviation = json.load(json_data)
        
        # 성경이름정보를 정의한 JSON파일에서 요청 성경 약어로부터 대한성서공회 성경영문약어 획득하여 질의정보에 정의
        bible_name_info = biblical_abbreviation[self.bible_name_abbr_kr]

        # 현재 셩 이름경정보를 클래스 속성 정보에 할당
        ReadingBible.bible_info['class'] = bible_name_info['class']
        ReadingBible.bible_info['bible_name_kr'] = bible_name_info['bible_name_kr']
        ReadingBible.bible_info['bible_name_abbr_kr'] = self.bible_name_abbr_kr
        ReadingBible.bible_info['bible_name_en'] = bible_name_info['bible_name_en']
        ReadingBible.bible_info['bible_name_abbr_en'] = bible_name_info['bible_name_abbr_en']
        
        # 질의정보에 성경구절 질의 정보 정의
        query_info['bskorea_name'] = bible_name_info['bskorea__name']
        query_info['chapter'] = self.chapter
        query_info['from_section'] = self.from_section
        query_info['to_section'] = self.to_section
        
        return query_info
        
    def create_bible_chapter(self, query_info):
        '''
        대한 성서 공회 개역한글 성경을 셩경별 장단위로 크롤링하여 구절정보을 포함하는 딕셔너리 생성
        '''
        # 대한성서공회  url setting을 위한 필요정보
        #version = 'HAN'         # 개역한글
        #book = 'gen'            # 창세기
        #chapter = 1             # 1장을 가져온다.
        
        # 목표 url
        # url = "http://www.bskorea.or.kr/bible/korbibReadpage.php?version=HAN&book=gen&chap=1"
        # 웹 서버에 요청 보내고 응답 받기
        resp = requests.get("http://www.bskorea.or.kr/bible/korbibReadpage.php?version=HAN&book={}&chap={}".format(query_info['bskorea_name'], query_info['chapter']))
        # 전송된 HTML 문서 얻기
        html = resp.text
        # HTML 문서를 DOM으로 변환하기
        soup = BeautifulSoup(html, "lxml")
        # 목표 HTML 요소 선택(DIV tag중 ID가 tdBible1인 요소의 하위에 SPAN tag 모두)
        html_element = soup.find("div", {"id" : "tdBible1"}).find_all("span")

        # 크롤링한 성경 장의 구절정보을 담을 딕셔너리 선언
        bible_chapter_table = {}
 
        # 목표 텍스트 추출
        for bible_chapter_verses in html_element:
            # HTML DIV tag중 ID가 tdBible1인 요소의 하위에 SPAN tag 모두를 가지고 와서
            # SPAN tag의 텍스트 성경 구절들을 List객체로 만듬
            table0 = bible_chapter_verses.get_text().strip().split("  ")
            # List객체중 " \xa0\xa0\xa0"로 좌우 분리
            table1 = table0[0].split(" \xa0\xa0\xa0")
        
            # List객체중 구절 번호와 내용의 쌍으로 된것만 선택
            if len(table1) > 1:
                # List객체중 ""\n\n"가 들어 있는것을 좌우 분리하여 죄측내용만 선택
                tmp = table1[1].rsplit("\n\n")
                if len(tmp) > 1:
                    verses = tmp[0]
                else:
                    verses = table1[1]
        
                bible_chapter_table[table1[0]] = verses

        return bible_chapter_table

    def get_bible_paragraph(self, bible_paragraph_info):
        '''
        성경구절 내용 획득하여 Return
        '''
        # 셩경구절정보 초기화
        self.bible_name_abbr_kr = ""
        self.chapter = ""
        self.from_section = ""
        self.to_section = ""
        
        # 셩경구절내용
        response_value = ""
        
        self.set_bible_paragraph_info(bible_paragraph_info)
        query_info = self.get_query_info()
        bible_chapter_contents = self.create_bible_chapter(query_info)
        self.logger.debug("success creating bible_paragraph_contents")
        
        if query_info['to_section']:
            start_verses = int(query_info['from_section'])
            end_verses = int(query_info['to_section'])
            for i in range(start_verses, end_verses+1):
                response_value += " "
                response_value += str(i)
                response_value += ". "
                response_value += bible_chapter_contents[str(i)]
        else:
            response_value += query_info['from_section']
            response_value += ". "
            response_value += bible_chapter_contents[query_info['from_section']]
            
        # 현재 셩경장절과 내용정보를 클래스 속성 정보에 할당
        ReadingBible.bible_info['bskorea_name'] = query_info['bskorea_name']
        ReadingBible.bible_info['chapter'] = query_info['chapter']
        ReadingBible.bible_info['from_section'] = query_info['from_section']
        ReadingBible.bible_info['to_section'] = query_info['to_section']
        ReadingBible.bible_chapter_contents = bible_chapter_contents
        
        return response_value.strip()
